THINGS TO KEEP IN MIND / IMPROVE UPON:

major:

- for the extra feature engineering part with the histones check whether use correct col for histone peaks and consider incorporating the other extra columns as info as well
- check if the column definitons are even correct !!!
- keep gene names to make sure that we are not joining the wrong thing!!!
- what data is this, asking for referecne 
- use DNase data, and use histone mark data from H3K4me3 and HЗК27ас since their peak density plots clearly indicate nice normal dist and peaks around the TSS, which indicates that the region is active and ready for transcription

- add standarization within cell line before stacking the data (can test different types of standardization)
- in end add training with all data but test to pred !!!

- for now just stack the data of the two cell lines and train model using combined data with the hope of fitting a model that generalizes beyond cell line
-> just stacking the data might not be great
- think about trying signal processing tricks to extend features using ML4H libray, on top of self designed features
- test simple implementation of CNN (look int literature for what makes sense, see lectures for some general steps) + on top use surrogate spearman correlation loss to train the CNN
-> make sure to not give CNN too many par
- try getting pairwise loss surrogate for spearman corr to work (but prob have to then go with pytorch), however is approx only and can get quite comp expensive for large datasets due to pairwise comparison
- right now use simple setup and dont use all the data that we could be using
-> once have det optimal model with optimal hyperpar train on all train & val data and use for final prediction

minor:
- dropped the gene names, so double check if the gene ordering aligns in the x and y data


In [1]:
import pandas as pd
import pybedtools
from scripts.gen import tsvLoader, bedLoader

# Specify the columns of interest for the TSS data
cols_to_use = ['chr', 'TSS_start', 'TSS_end', 'gene_name', 'gene_start', 'gene_end', 'strand']

# Load TSS data and histone mark data as BedTool objects
tss_x_1_train = tsvLoader(path='data/CAGE-train/X1_train_info.tsv', cols_to_use=cols_to_use)  # Returns BedTool object
hist_H3K4me3 = bedLoader(path="data/H3K4me3-bed/X1.bed")  # Returns BedTool object

# Step 1: Find the closest peak to each TSS entry
# This will add information about the closest peak to each TSS
closest_peaks = tss_x_1_train.closest(hist_H3K4me3, d=True)

# Step 2: Convert to a pandas DataFrame for further processing
df_closest = closest_peaks.to_dataframe(
    names=[
        'chr', 'TSS_start', 'TSS_end', 'gene_name', 'gene_start', 'gene_end', 'strand',
        'peak_chr', 'peak_start', 'peak_end', 'peak_name', 'peak_score', 'peak_strand',
        'peak_signal', 'peak_p_value', 'peak_q_value', 'peak_read_count', 'distance'
    ]
)

# Step 3: Calculate the closest peak's strength
# The strength can be captured by the 'peak_signal' column, which represents the score/intensity of the closest peak.
df_closest['closest_peak_strength'] = df_closest['peak_signal']

# Step 4: Count the number of peaks within a 2kb window around each TSS
# Use `window` to find peaks within 2kb (2000 bases) of each TSS
window_peaks = tss_x_1_train.window(hist_H3K4me3, w=2000)

# Convert the window results into a DataFrame
df_window = window_peaks.to_dataframe(
    names=[
        'chr', 'TSS_start', 'TSS_end', 'gene_name', 'gene_start', 'gene_end', 'strand',
        'peak_chr', 'peak_start', 'peak_end', 'peak_name', 'peak_score', 'peak_strand',
        'peak_signal', 'peak_p_value', 'peak_q_value', 'peak_read_count'
    ]
)

# Step 5: Aggregate the number of peaks and the average peak strength for each TSS within the window
# Group by TSS and calculate count and mean of 'peak_signal'
peak_summary = df_window.groupby(['chr', 'TSS_start', 'TSS_end', 'gene_name']).agg(
    number_of_peaks=('peak_signal', 'count'),
    average_peak_strength=('peak_signal', 'mean')
).reset_index()

# Merge the closest peak information with the summary data from the window operation
result_df = pd.merge(
    df_closest[['chr', 'TSS_start', 'TSS_end', 'gene_name', 'closest_peak_strength', 'distance']],
    peak_summary,
    on=['chr', 'TSS_start', 'TSS_end', 'gene_name'],
    how='left'
)

# Fill NaN values for TSS entries with no nearby peaks (e.g., for average peak strength)
result_df['number_of_peaks'].fillna(0, inplace=True)
result_df['average_peak_strength'].fillna(0, inplace=True)

# Display the resulting DataFrame with the new features
print(result_df.head())

# Save the extended TSS data with the new features (optional)
result_df.to_csv('data/CAGE-train/X1_train_info_extended.tsv', sep='\t', index=False)


     chr  TSS_start  TSS_end gene_name  closest_peak_strength  distance  \
0  chr10      49219    49269     TUBB8               5.178203     24539   
1  chr10     135483   135533   ZMYND11               8.986245        32   
2  chr10     689618   689668     DIP2C               4.990885    162107   
3  chr10     931427   931476    LARP4B              10.234297        64   
4  chr10    1005837  1005887    GTPBP4              10.776866     14303   

   number_of_peaks  average_peak_strength  
0              0.0               0.000000  
1              3.0              11.824290  
2              0.0               0.000000  
3              2.0               8.168684  
4              0.0               0.000000  


/var/folders/tp/l4sw2ybd7t3b9gbsrzngsn6w0000gn/T/ipykernel_39413/231959165.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df['number_of_peaks'].fillna(0, inplace=True)
/var/folders/tp/l4sw2ybd7t3b9gbsrzngsn6w0000gn/T/ipykernel_39413/231959165.py:59: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [2]:
result_df

,chr,TSS_start,TSS_end,gene_name,closest_peak_strength,distance,number_of_peaks,average_peak_strength
0,chr10,49219,49269,TUBB8,5.178203,24539,0.0,0.000000
1,chr10,135483,135533,ZMYND11,8.986245,32,3.0,11.824290
2,chr10,689618,689668,DIP2C,4.990885,162107,0.0,0.000000
3,chr10,931427,931476,LARP4B,10.234297,64,2.0,8.168684
4,chr10,1005837,1005887,GTPBP4,10.776866,14303,0.0,0.000000
...,...,...,...,...,...,...,...,...
14305,chr9,137575166,137575216,DPH7,2.578880,63,2.0,9.372441
14306,chr9,137590440,137590490,ZMYND19,19.090852,190,2.0,17.529227
14307,chr9,137614164,137614214,ARRDC1,17.730647,3213,0.0,0.000000
14308,chr9,137618991,137619041,EHMT1,17.730647,107,2.0,17.347564


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
import numpy as np

In [3]:
# functions used


# extends cell line data with simple features
def extend_df(df, col_to_keep):
    # Add a binary column for strand
    df['strand_binary'] = df['strand'].map({'+': 1, '-': 0})
    
    # gene length
    df['gene_length'] = df['gene_end'] - df['gene_start']
    
    # transcription site length
    df['trans_site_len'] = df['TSS_end'] - df['TSS_start']
    
    # ratio transcription site length & land gene length
    df['trans_gene_ratio'] = df['trans_site_len'] / df['gene_length']
    
    return df[col_to_keep]



# NOT BEING USED !!!!
# Custom surrogate loss function for Spearman correlation (Pairwise loss)
def pairwise_loss(y_true, y_pred):
    # Convert predicted values to a 1D numpy array
    y_pred = y_pred.ravel()
    
    # Calculate the pairwise differences for predicted values
    pred_diff = np.expand_dims(y_pred, axis=1) - np.expand_dims(y_pred, axis=0)
    
    # Calculate the pairwise differences for true values
    true_diff = np.expand_dims(y_true, axis=1) - np.expand_dims(y_true, axis=0)
    
    # Sigmoid of the differences (this creates a smooth approximation for ranking)
    sigmoid_pred_diff = 1 / (1 + np.exp(-pred_diff))
    
    # Compute the gradient as the difference between the predictions and the true order
    grad = sigmoid_pred_diff - (true_diff > 0).astype(np.float32)
    
    # The Hessian is a constant in pairwise loss (can be set to a fixed value)
    hess = np.ones_like(grad)
    
    return grad.ravel(), hess.ravel()


# Custom evaluation metric for Spearman correlation
# -> used as eval metric during training ()
def spearman_correlation(preds, train_data):
    y_true = train_data.get_label()
    corr, _ = spearmanr(y_true, preds)
    return 'spearman', corr, True



In [4]:
# parameters


# reproducibility seed
seed = 42

hpo_size = 0.2


In [ ]:
# load data


# load cell line data (x/y for feature/target, 1/2/3 for cell line and train/val/test indicating assignment)

# features 
x_1_train = pd.read_csv('data/CAGE-train/X1_train_info.tsv', sep='\t')
x_1_val = pd.read_csv('data/CAGE-train/X1_val_info.tsv', sep='\t')
x_2_train = pd.read_csv('data/CAGE-train/X2_train_info.tsv', sep='\t')
x_2_val = pd.read_csv('data/CAGE-train/X2_val_info.tsv', sep='\t')

x_3_test = pd.read_csv('data/CAGE-train/X3_test_info.tsv', sep='\t')

# targets
y_1_train = pd.read_csv('data/CAGE-train/X1_train_y.tsv', sep='\t')
y_1_val = pd.read_csv('data/CAGE-train/X1_val_y.tsv', sep='\t')
y_2_train = pd.read_csv('data/CAGE-train/X2_train_y.tsv', sep='\t')
y_2_val = pd.read_csv('data/CAGE-train/X2_val_y.tsv', sep='\t')


# save gene order (might be relevant down the line)
genes_1_train = x_1_train['gene_name']
genes_1_val = x_1_val['gene_name']
genes_2_train = x_2_train['gene_name']
genes_2_val = x_2_val['gene_name']
genes_3_test = x_3_test['gene_name']


# features to use for prediction
pred_feat = ['strand_binary', 'gene_length', 'trans_site_len', 'trans_gene_ratio']

''' 
1) To find the optimal model and hyperparameters we use the train data (split for training and hyperparoptim)
2) Once we have found the optimal model we get an estimate for the test set error on the validation data
3) Before submission we train the best model (model + hyperpar) on the train and validation data and then predict test for submission
'''

In [6]:
# data preprocessing & feature engineering

# ...
x_1_train = extend_df(x_1_train, pred_feat)
x_1_val = extend_df(x_1_val, pred_feat)
x_2_train = extend_df(x_2_train, pred_feat)
x_2_val = extend_df(x_2_val, pred_feat)

x_3_test = extend_df(x_3_test, pred_feat)


# drop gene name from y
y_1_train = y_1_train.drop('gene_name', axis=1)
y_1_val = y_1_val.drop('gene_name', axis=1)
y_2_train = y_2_train.drop('gene_name', axis=1)
y_2_val = y_2_val.drop('gene_name', axis=1)


# stack features of cell line 1 and 2 (STARTING POINT, CAN TEST SOMETHING ELSE TO IMPROVE UPON THIS)
# CAREFUL, remember stacking order for gene names if necessary!!!

# x
x_1_2_train = pd.concat([x_1_train, x_2_train], ignore_index=True)
x_1_2_val = pd.concat([x_1_val, x_2_val], ignore_index=True)
# y
y_1_2_train = pd.concat([y_1_train, y_2_train], ignore_index=True)
y_1_2_val = pd.concat([y_1_val, y_2_val], ignore_index=True)


# split train data into data for training and hyperparoptim
x_1_2_train_only, x_1_2_train_hpo, y_1_2_train_only, y_1_2_train_hpo = train_test_split(x_1_2_train, y_1_2_train, test_size=hpo_size, random_state=seed)

In [ ]:
# Data to train LightGBM model
x_y_1_2_train_only = lgb.Dataset(x_1_2_train_only, label=y_1_2_train_only)

# Parameters for LightGBM with Huber loss
params = {
    'boosting_type': 'gbdt',
    'objective': 'huber',
    'alpha': 0.9,
    'learning_rate': 0.05,
    'num_leaves': 40,
    'n_estimators': 100
}

# train model
model = lgb.train(
    params,
    x_y_1_2_train_only,
)

# eval model for hyperparoptim
y_pred_hpo = model.predict(x_1_2_train_hpo)

# Calculate Spearman correlation for the predictions
spearman_corr, _ = spearmanr(y_1_2_train_hpo, y_pred_hpo)
print(f'Spearman correlation on validation set (x_y_1_2_train_hpo): {spearman_corr}')
# -> use this for hyperparoptim

# FINAL ESTIMATE TEST SET ERROR ... (use val data, but in end use test data)

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [ ]:
# TODO: 
# Load your feature (bed and/or bigwig and/or fasta) and target files (tsv) here.
# Decide which features to use for training. Feel free to process them however you need.

# NOTE: 
# bed and bigwig files contain signals of all chromosomes (including sex chromosomes).
# Training and validation split based on chromosomes has been done for you. 
# However, you can resplit the data in any way you want.

path_data = "/path/to/your/data/files"  # TODO
path_test = "/path/to/test/info/file"   # X3_test_info.tsv ; TODO
test_genes = pd.read_csv(path_test, sep='\t')
# ---------------------------INSERT CODE HERE---------------------------




# ---------------------------------------------------------------------- 

## Work Package 1.2 - Model Building

In [ ]:
# TODO: 
# Select the best model to predict gene expression from the obtained features in WP 1.1.

# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [ ]:
# TODO:
# Using the model trained in WP 1.2, make predictions on the test data (chr 1 of cell line X3).
# Store predictions in a variable called "pred" which is a numpy array.

pred = None
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'path/to/save/output/file'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "LastName_FirstName_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes['gex_predicted'] = pred.tolist()
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)